# Решение
    

In [ ]:
import torch
import timm
import torchvision.transforms as transforms
from PIL import Image
import torch.nn.functional as F
import numpy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

COLORS = ["bezhevyi", "belyi", "biryuzovyi", "bordovyi", "goluboi", "zheltyi", "zelenyi", "zolotoi", "korichnevyi", "krasnyi", "oranzhevyi", "raznocvetnyi", "rozovyi", "serebristyi", "seryi", "sinii", "fioletovyi", "chernyi"]

model = timm.create_model("efficientnet_b3", pretrained=False, num_classes=len(COLORS), drop_rate=0.3)
model = model.to(device)

checkpoint = torch.load("general_3_reg.pth", map_location=device)
model.load_state_dict(checkpoint)
model.eval()

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


def detect_color(image_path):
    """
    Загружает фото товара, применяет необходимые преобразования
    и возвращает 5 лучших прогнозируемых цветов для него и еще возвращает
    18 лучших Распознанных цветов с вероятностями .
    """
     

    img = Image.open(image_path).convert("RGB")
    img_tensor = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(img_tensor)
        probabilities = F.softmax(outputs, dim=1)

        top19_vals, top19_idx = torch.topk(probabilities, len(COLORS))
        color_probabilities_19 = {
            COLORS[idx.item()]: format(top19_vals[0][i].item(), ".4f") for i, idx in enumerate(top19_idx[0])
        }

        top5_vals, top5_idx = torch.topk(probabilities, 5)
        color_probabilities_5 = {
            COLORS[idx.item()]: format(top5_vals[0][i].item(), ".4f") for i, idx in enumerate(top5_idx[0])
        }

    return color_probabilities_19, color_probabilities_5  

path = input("Запишите локальный путь к фото: ")
color_probs_19, color_probs_5 = detect_color(path)  

print("Товар имеет цвет:", list(color_probs_5.keys())[0])  # Most probable color
print("\nТоп-5 наиболее вероятных цветов: ", color_probs_5)
print("\nТоп-19 наиболее вероятных цветов: ", color_probs_19)


Товар имеет цвет: fioletovyi

Топ-5 наиболее вероятных цветов:  {'fioletovyi': '0.9922', 'raznocvetnyi': '0.0038', 'rozovyi': '0.0016', 'bordovyi': '0.0008', 'sinii': '0.0004'}

Топ-19 наиболее вероятных цветов:  {'fioletovyi': '0.9922', 'raznocvetnyi': '0.0038', 'rozovyi': '0.0016', 'bordovyi': '0.0008', 'sinii': '0.0004', 'krasnyi': '0.0003', 'belyi': '0.0002', 'chernyi': '0.0002', 'zelenyi': '0.0002', 'seryi': '0.0001', 'biryuzovyi': '0.0000', 'oranzhevyi': '0.0000', 'korichnevyi': '0.0000', 'bezhevyi': '0.0000', 'zheltyi': '0.0000', 'zolotoi': '0.0000', 'goluboi': '0.0000', 'serebristyi': '0.0000'}
